# Exercise 5: Inversion theory: Optimal Estimation Method (OEM)

Manfred Brath, Oliver Lemke

In [ ]:
%matplotlib widget

import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import colormaps as cm
import numpy as np
from scipy.linalg import inv

from pyarts import xml
from oem import forward_model

os.makedirs("plots", exist_ok=True)

In this exercise you will work with "realistic" data measured by a water  
vapor radiometer. The data is not real but has been simulated for a well-  
known atmospheric state using ARTS. Simulated measurements allow to  
compare retrieval results to the true atmospheric state. The radiometer  
(image below) measures thermal radiation in a frequency range around the  
$22\,\text{GHz}$ water vapor absorption line.   
As the pressure broadening of absorption lines varies with height the  
measurement contains information about the vertical water vapor profile.  
This information can be retrieved using the "Optimal Estimation Method" (OEM).   
The radiometer is placed in $10\,\text{km}$ height, which resembles an upward  
looking airborne measurement. The scarce concentration of water vapor in the   
stratosphere allows to perform a linear retrieval approach. Retrievals that  
cover the whole atmosphere, including the highly absorbent lower troposphere,  
need more advanced retrieval approaches like an iterative OEM. 

![radiometer](H2Orad.jpg)

* Run the next cell.

In [ ]:
# Load the (simulated) measurement.
measurement = xml.load("input/measurement.xml")
f_grid = measurement.grids[0]
y_measurement = measurement.data

# Load the a priori atmospheric state.
atm_fields = xml.load("input/x_apriori.xml")
z = atm_fields.get("z", keep_dims=False)
x_apriori = atm_fields.get("abs_species-H2O", keep_dims=False)

# Load nd set the covariance matrices.
S_xa = xml.load("input/S_xa.xml")
S_y = 2.5e-3 * np.eye(f_grid.size)  # in [K^2]

* Plot the observed brightness
temperature spectrum `y_measurement` as function of frequency
`f_grid`.

In [ ]:
# Plot the y measurement.
fig, ax = plt.subplots(1,1)
ax.plot(f_grid/1e9, y_measurement, label="y measurement")

* Run the next cell to simulate the brightness temperature spectrum 
`y` and the water vapor Jacobian `K` for  
the *a priori* state.

In [ ]:
# Run the forward model (ARTS).
y, K = forward_model(f_grid, atm_fields)

* Plot the simulated brightness temperature spectrum alongside with
the observed brightness temperature spectrum.

In [ ]:
# Plot the y measurement alongside the simulated y for the a priori.
ax.plot(f_grid/1e9, y, label="y forward model")
ax.set_xlabel("Frequency [GHz]")
ax.set_ylabel("Brightness temperature [K]")
ax.legend()
fig.tight_layout()

* Plot the Jacobians `K` in a suitable way. Explain the plot.

In [ ]:
# Plot the Jacobians.

width_in_cm = 29.7
height_in_cm = 20.9

fig1, ax1 = plt.subplots(1,1)
fig1.set_size_inches(width_in_cm/2.54,h=height_in_cm/2.54)
pcm=ax1.pcolormesh(f_grid/1e9, z/1e3, K.T, shading="auto")
fig1.colorbar(pcm, ax=ax1, label=r"$\frac{\partial F_i}{\partial q_j}$  K vmr$_{H_2O}^{-1}$")
ax1.set_xlabel("Frequency [GHz]")
ax1.set_ylabel("Altitude [km]")
ax1.set_title("Jacobian")
fig1.tight_layout()

* Plot the measurement covariance matrix `S_y` and the *apriori*  covariance matrix `S_xa` in a suitable way.  
What do the covariance matrices mean?

In [ ]:
# Plot the covariance matrices.
width_in_cm = 29.7
height_in_cm = 20.9

fig2, ax2 = plt.subplots(1,2,sharey=False, sharex=False)
fig2.set_size_inches(width_in_cm/2.54,h=height_in_cm/2.54)

pcm = ax2[0].pcolormesh(f_grid/1e9, f_grid/1e9, S_y, shading="auto")
fig2.colorbar(pcm, ax=ax2[0], label="$S_y$ / $K^2$", shrink = 0.5)
ax2[0].set_xlabel("Frequency [GHz]")
ax2[0].set_ylabel("Frequency [GHz]")
ax2[0].set_title("Covariance matrix $S_y$")
ax2[0].set_aspect('equal', 'box')

pcm = ax2[1].pcolormesh(z/1e3, z/1e3, S_xa, shading="auto")
fig2.colorbar(pcm, ax=ax2[1], label="$S_{xa}$ / vmr$^2$", shrink = 0.5)
ax2[1].set_xlabel("Altitude [km]")
ax2[1].set_ylabel("Altitude [km]")
ax2[1].set_title("Covariance matrix $S_{xa}$")
ax2[1].set_aspect('equal', 'box')

fig2.tight_layout()




* Implement the function `retrieve()` according to the OEM solution:  
$$\hat{\mathbf{x}}=\mathbf{x}_{a}+\left(\mathbf{K}^{T}\mathbf{S}_{y}^{-1}\mathbf{K}+\mathbf{S}_{xa}^{-1}\right)^{-1}\mathbf{K}^{T}\mathbf{S}_{y}^{-1}\left(\mathbf{y}_{measure}-\mathbf{y}_{a}\right)$$

 with $\mathbf{x}_{a}$ the a priori profile, $\mathbf{K}$ the Jacobian,
$\mathbf{S}_{y}$ the measurement covariance matrix, $\mathbf{S}_{xa}$
the *a priori* covariance matrix, $\mathbf{y}_{measure}$ the
observed brightness temperature spectrum and $\mathbf{y}_{a}$ the
simulated brightness temperature spectrum of profile $\mathbf{x}_{a}$.  
In Python, a matrix `M` can be transposed using `M.T`
and inversed using `inv(M)` We are using the inverse function 
`scipy.linalg.inv()` provided by the SciPy package. 
Two matrices `M1` and `M2` can be multiplied using
`M1 @ M2.`

* Use the function `retrieve()` to retrieve the water vapor profile.

In [ ]:
# Implement the retrieve function.
def retrieve(y, K, xa, ya, Sa, Sy):
    """Perform an OEM retrieval.

    Parameters:
        y (np.ndarray): Measuremed brightness temperature [K].
        K (np.ndarray): Jacobians [K/1].
        xa (np.ndarray): A priori state [VMR].
        ya (np.ndarray): Forward simulation of a priori state ``F(xa)`` [K].
        Sa (np.ndarray): A priori error covariance matrix.
        Sy (np.ndarray): Measurement covariance matrix

    Returns:
        np.ndarray: Retrieved atmospheric state.

    """
    print("Function needs to be implemented by you")

    

    return xa + inv(K.T @ inv(Sy) @ K + inv(Sa)) @ K.T @ inv(Sy) @ (y - ya)


In [ ]:
# retrieve
x_oem = retrieve(y_measurement, K, x_apriori, y, S_xa, S_y)


* Plot the retrieved water vapor `x_oem` and the *a priori* 
water vapor profile as function of height `z`.

* Load the true water vapor retrieval (`input/x_true.xml`) and
add it to the previous plot. Dicuss the results.

In [ ]:
# Plot the OEM result next to the true atmospheric state and the a priori.
fig3, ax3 = plt.subplots(1,1)
ax3.plot(x_apriori, z, label="a priori")
ax3.plot(x_oem, z, label="OEM") 

ax3.set_xlabel("H2O [VMR]")
ax3.set_ylabel("Altitude [m]")
ax3.set_title("Retrieved atmospheric state")    

atm_true = xml.load("input/x_true.xml")
x_true = atm_true.get("abs_species-H2O", keep_dims=False)
ax3.plot(x_true, z, label="true")
ax3.legend()

* Implement the function `averaging_kernel_matrix()` to calculate
the same-named matrix:  
$$\mathbf{A}=\left(\mathbf{K}^{T}\mathbf{S}_{y}^{-1}\mathbf{K}+\mathbf{S}_{xa}^{-1}\right)^{-1}\mathbf{K}^{T}\mathbf{S}_{y}^{-1}\mathbf{K}$$

In [ ]:
# Implement the averaging_kernel_matrix function.
def averaging_kernel_matrix(K, Sa, Sy):
    """Calculate the averaging kernel matrix.

    Parameters:
        K (np.ndarray): Simulated Jacobians.
        Sa (np.ndarray): A priori error covariance matrix.
        Sy (np.ndarray): Measurement covariance matrix.

    Returns:
        np.ndarray: Averaging kernel matrix.
    """
    #print("Function needs to be implemented by you")

    return inv(K.T @ inv(Sy) @ K + inv(Sa)) @ K.T @ inv(Sy) @ K
    

In [ ]:
# Calculate averaging kernel 
A = averaging_kernel_matrix(K, S_xa, S_y)

* Plot the kernels (columns) of $\mathbf{A}$ as function of height
`z` and interpret the results.  
The measurement response is defined as the sum over all averaging
kernels in a given height (row). The measurement response indicates
in which heights the measurement actually adds information to the
retrieval result.
* Calculate the measurement response and plot it together with the averaging
kernels.
* In which heights does the measurement provide useful information?
* Is it possible to estimate the vertical resolution?

In [ ]:
# Plot averaging kernels

width_in_cm = 20.9
height_in_cm = 29.7

fig = plt.figure(constrained_layout=True)
fig.set_size_inches(width_in_cm / 2.54, height_in_cm / 2.54)
ax = gridspec.GridSpec(ncols=2, nrows=3, figure=fig, hspace=0.15)

ax1 = fig.add_subplot(ax[0:2,0:])
pcm=ax1.pcolormesh(z/1e3,z/1e3,A, shading="auto", cmap="plasma")
fig.colorbar(pcm, ax=ax1, label="vmr / vmr")
ax1.set_xlabel("Altitude [km]")
ax1.set_ylabel("Altitude [km]")    
ax1.axis('equal')


cmap =cm['plasma']
colors = cmap(np.linspace(0, 1, len(z)))
ax2 = fig.add_subplot(ax[2,0])
for i in range(len(z)):
    ax2.plot(A[i,:],z/1e3,color=colors[i])
ax2.set_title(r"Sensitivity  (rows of $\mathbf{A}$)")
ax2.set_xlabel("vmr / vmr")
ax2.set_ylabel("Altitude [km]")

# add colorbar indicating the altitude
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=len(z)))
cbar = fig.colorbar(sm, ax=ax2, orientation='vertical', pad=0)
cbar.set_label('Altitude index')


ax3 = fig.add_subplot(ax[2,1])
for i in range(len(z)):
    ax3.plot(A[:,i],z/1e3,color=colors[i])
ax3.set_title(r"Contribution  (columns of $\mathbf{A}$)") 
ax3.set_xlabel("vmr / vmr")
ax3.set_ylabel("Altitude [km]")  

measurement_response = np.sum(A,0)
ax3.plot(measurement_response/10, z/1e3, color="black", linewidth=2, label="Measurement\n response/10")
ax3.legend()


In [ ]:
#Estimate degrees of Freedom
dof = np.trace(A)
print(f"Degrees of Freedom: {dof:.2f}")

#Estimate entropy
I=np.eye(A.shape[0])
entropy = - 0.5 * np.log(np.linalg.det(I - A))
print(f"Entropy: {entropy:.2f}")

In [ ]:
# Implement the averaging_kernel_matrix function.
def gain_matrix(K, Sa, Sy):
    """Calculate the averaging kernel matrix.

    Parameters:
        K (np.ndarray): Simulated Jacobians.
        Sa (np.ndarray): A priori error covariance matrix.
        Sy (np.ndarray): Measurement covariance matrix.

    Returns:
        np.ndarray: Averaging kernel matrix.
    """
    #print("Function needs to be implemented by you")

    return inv(K.T @ inv(Sy) @ K + inv(Sa)) @ K.T @ inv(Sy)
    

In [ ]:
# Calculate averaging kernel 
G = gain_matrix(K, S_xa, S_y)

In [ ]:
width_in_cm = 29.7
height_in_cm = 20.9

fig = plt.figure(constrained_layout=True)
fig.set_size_inches(width_in_cm / 2.54, height_in_cm / 2.54)
ax = gridspec.GridSpec(ncols=2, nrows=2, figure=fig, hspace=0.35)


ax11 = fig.add_subplot(ax[0, :])
pcm=ax11.pcolormesh(f_grid/1e9,z/1e3,G, cmap="plasma")
fig.colorbar(pcm, ax=ax11, label="vmr/K")
ax11.set_title("Gain matrix G")
ax11.set_xlabel("Frequency [GHz]")
ax11.set_ylabel("Altitude [km]")

f_indices = np.arange(0, f_grid.size, step=300)
cmap = cm.get_cmap("plasma")
cmap = cmap(np.linspace(0.2, 0.9, len(f_indices)))
ax10 = fig.add_subplot(ax[1, 0])     
for i, fi in enumerate(f_indices):
    ax10.plot(G[:,fi],z/1e3, color=cmap[i], label=f"{f_grid[fi]/1e9:.3f} GHz")
ax10.set_xlabel("vmr/K")
ax10.set_ylabel("Altitude [km]")
ax10.set_title(r"Gain matrix $\mathbf{G}$ - Contribution functions")       
ax10.legend(title="Frequency")

z_indices = np.arange(0, z.size, step=5)
cmap = cm.get_cmap("plasma")
cmap = cmap(np.linspace(0.2, 0.9, len(z_indices)))
ax11 = fig.add_subplot(ax[1, 1])
for i, zi in enumerate(z_indices):
    ax11.plot(f_grid/1e9, G[zi,:], color=cmap[i], label=f"{z[zi]/1e3:.1f} km")
ax11.set_xlabel("Frequency [GHz]")
ax11.set_ylabel("vmr/K")
ax11.set_title(r"Gain matrix $\mathbf{G}$ - Sensitivity")       
ax11.legend(title="Altitude")

